# Serialization Builder

After an ontology model is developed, there needs to be a method of uploading data that fits the model in question. 

This data load process is often fed by flat or tree-like datafiles - perhaps simple CSV, XML or JSON documents. 

A serialization helps marshall structured data into a given ontologically defined graph format. 

The serialization has a name, and contains multiple mappings. 

Each mapping describes some link between named attributes in the input file, and their equivalent ontological markup. Some inputs describe entities of a given type, while others describe relations between these entities, and others describe simple data-properties those entities might have. 



In [29]:
import owlready2 as owlr

import xml.etree.ElementTree as ET

In [10]:
sample_onto_file = "../sample_ontology.owl"
sample = owlr.get_ontology(sample_onto_file).load()

In [16]:
sample_ser_file = "../Serialization.owl"
serial = owlr.get_ontology(sample_ser_file).load()

* Owlready2 * WARNING: ObjectProperty http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MappingMetaTarget belongs to more than one entity types: [owl.AnnotationProperty, owl.ObjectProperty, owl.topObjectProperty]; I'm trying to fix it...


In [17]:
dir(sample)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abbreviate',
 '_add_annotation_axiom',
 '_add_data_triple_raw_spod',
 '_add_data_triple_raw_spodsd',
 '_add_data_triple_spod',
 '_add_obj_triple_raw_spo',
 '_add_obj_triple_spo',
 '_base_iri',
 '_bnodes',
 '_del_annotation_axiom',
 '_del_data_triple_raw_spod',
 '_del_data_triple_raw_spodsd',
 '_del_data_triple_spod',
 '_del_list',
 '_del_obj_triple_raw_spo',
 '_del_obj_triple_spo',
 '_destroy_cached_entities',
 '_entity_destroyed',
 '_get_annotation_axioms',
 '_get_data_triple_sp_od',
 '_get_data_triples_s_pod',
 '_get_data_triples_sp_od',
 '_get_data_triples_sp

In [52]:
list(serial.properties())

[Serialization.GeneratedOn,
 Serialization.MappingDomain,
 Serialization.MappingLabel,
 Serialization.MappingRange,
 Serialization.MappingSubType,
 Serialization.SerializationLabel,
 Serialization.SerializationParentLabel,
 Serialization.UniqueIdentifier,
 Serialization.MappingMetaTarget,
 Serialization.ContainsMapping,
 Serialization.MappingMemberOf,
 Serialization.MappingMetaTarget]

In [21]:
serial.MetaClass.iri

'http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MetaClass'

In [19]:
list(sample.classes()), list(sample.object_properties()), list(sample.data_properties())

([sample_ontology.SomeClass],
 [sample_ontology.someProperty],
 [sample_ontology.someDataProperty])

In [62]:
def generate_meta_definitions(onto):
    meta_defs = []
    for mc in [(serial.MetaClass, onto.classes), (serial.MetaProperty, onto.object_properties), (serial.MetaDataProperty, onto.data_properties)]:
        for c in mc[1]():
            meta_defs.append((c.iri,mc[0].iri))
    return meta_defs
        

In [63]:
generate_meta_definitions(sample)

[('http://www.semanticweb.org/tomk/ontologies/2023/6/sample#SomeClass',
  'http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MetaClass'),
 ('http://www.semanticweb.org/tomk/ontologies/2023/6/sample#someProperty',
  'http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MetaProperty'),
 ('http://www.semanticweb.org/tomk/ontologies/2023/6/sample#someDataProperty',
  'http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MetaDataProperty')]

In [68]:
def generate_xml_for_serialization(namespaces):
    X = ET.Element('rdf:RDF')
    default_namespace='http://www.company.com'
    for prefix, uri  in namespaces:
        X.set("xmlns:" + prefix, uri)
        
        
    ## Add block for serialization objects:
    for iri in [serial.Serialization.iri, 
              serial.Mapping.iri,
              serial.MetaClass.iri,             
              serial.MetaProperty.iri,
              serial.MetaDataProperty.iri]:
        q = ET.SubElement(X, "Class")
        q.set("rdf:about", iri)
        
    for iri in [ serial.ContainsMapping.iri,               
                 serial.MappingMetaTarget.iri,
                 serial.MappingDomain.iri,
                 serial.MappingRange.iri,
                 serial.MappingMetaTarget.iri,
                 serial.SerializationLabel.iri,
                 serial.SerializationParentLabel.iri]:
        q = ET.SubElement(X, "AnnotationProperty")
        q.set("rdf:about", iri)
    for iri,tp in generate_meta_definitions(sample):
        q = ET.SubElement(X, "NamedIndividual")
        q.set("rdf:about", iri)
        p = ET.SubElement(q, "rdf:type")
        p.set("rdf:resource", tp)
        
    ET.indent(X, space="\t", level=0)
    return ET.tostring(X).decode()

In [83]:
def create_mapping(name, target, ttype, properties=None):
    X = ET.Element("NamedIndividual")
    X.set("rdf:about", name)
    q = ET.SubElement(X,"rdf:type")
    q.set("rdf:resource", serial.Mapping.iri)
    q = ET.SubElement(X,"ser:MappingMetaTarget")
    q.set("rdf:resource", target)
    for k,v in properties.items():
        q = ET.SubElement(X,k)
        q.text =str(v)
    return X

In [84]:
ns_map = [('',"http://www.w3.org/2002/07/owl#"),
          ('xml:base',"http://www.w3.org/2002/07/owl"),
          ('rdf',"http://www.w3.org/1999/02/22-rdf-syntax-ns#"),
          ('xml:owl',"http://www.w3.org/2002/07/owl#"),
          ('dc',"http://purl.org/dc/elements/1.1/"),
          ('rdfs',"http://www.w3.org/2000/01/rdf-schema#")
         ]
print(generate_xml_for_serialization(ns_map))

<rdf:RDF xmlns:="http://www.w3.org/2002/07/owl#" xmlns:xml:base="http://www.w3.org/2002/07/owl" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:xml:owl="http://www.w3.org/2002/07/owl#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#">
	<Class rdf:about="http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#Serialization" />
	<Class rdf:about="http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#Mapping" />
	<Class rdf:about="http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MetaClass" />
	<Class rdf:about="http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MetaProperty" />
	<Class rdf:about="http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#MetaDataProperty" />
	<AnnotationProperty rdf:about="http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#ContainsMapping" />
	<AnnotationProperty rdf:about="http://www.semanticweb.org/tomk/ontologies/2022/11/ser

In [92]:
Q = create_mapping("mapping_test", "test" , "class", {"ser:SerializationLabel" : "Class",
                                                      "ser:SerializationParentLabel" : "ParentClass"})
ET.indent(Q, space="\t", level=0)
print(ET.tostring(Q).decode())

<NamedIndividual rdf:about="mapping_test">
	<rdf:type rdf:resource="http://www.semanticweb.org/tomk/ontologies/2022/11/serialization#Mapping" />
	<ser:MappingMetaTarget rdf:resource="test" />
	<ser:SerializationLabel>Class</ser:SerializationLabel>
	<ser:SerializationParentLabel>ParentClass</ser:SerializationParentLabel>
</NamedIndividual>
